# S2 TDS analysis

In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

In [2]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")


#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-04 18:09:05,0.0,-0.026423,1.190000e-07,4.560000e-08,5.273000e-08,4.984000e-07,0.037129,246.0,-1.683,4293.473,-0.001480,3.030000e-07,NaN,NaN
1,NaN,2023-05-04 18:09:14,9.0,-0.026250,1.160000e-07,4.490000e-08,5.173000e-08,4.984000e-07,0.037156,249.0,-1.683,4293.922,-0.001028,3.040000e-07,NaN,NaN
2,NaN,2023-05-04 18:09:22,17.0,-0.026029,1.140000e-07,4.400000e-08,5.075000e-08,4.972000e-07,0.037129,247.0,-1.683,4293.473,0.001907,3.040000e-07,NaN,NaN
3,NaN,2023-05-04 18:09:31,26.0,-0.026053,1.120000e-07,4.320000e-08,4.978000e-08,4.990000e-07,0.037163,248.0,-1.796,4293.922,-0.000418,3.040000e-07,NaN,NaN
4,NaN,2023-05-04 18:09:40,35.0,-0.024474,1.100000e-07,4.250000e-08,4.866000e-08,4.984000e-07,0.037160,248.0,-1.796,4293.585,-0.003055,3.050000e-07,NaN,NaN


In [3]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [4]:
#print hv comments
print(pd.unique(vaclog["Live comments"]))
vaclog_comm = vaclog.loc[:,["Time","Live comments"]]
vaclog_comm.dropna(inplace=True)
print(vaclog_comm)
vaclog_comm.index

[nan 'ega moved fully out friction desorption' 'refilling LN2'
 'LHe fill complete']
                    Time                            Live comments
6697 2023-05-05 10:07:35  ega moved fully out friction desorption
6698 2023-05-05 10:07:43                            refilling LN2
8354 2023-05-05 14:04:24                        LHe fill complete


Int64Index([6697, 6698, 8354], dtype='int64')

### Plotting

In [5]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [6]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='Dual exp')
plt.plot(vaclog["Time"],vaclog["helium"],marker=".", markersize=4,label='Dual exp')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S2 TDS Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')  

#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comm.index:
    plt.text(vaclog_comm["Time"][i],0.8,str(vaclog_comm["Live comments"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/TDS pressures.png')
plt.show()

In [7]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S2 TDS Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()